In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from binance.client import Client
import pandas as pd
import time

# Initialize Binance client (no API key needed for public data)
client = Client()

# Define your tickers - Binance uses different symbols, typically without '-USD'
tickers = ["BTCUSDT", "XRPUSDT", "ETHUSDT", "SOLUSDT", "DOGEUSDT", "ADAUSDT", 
           "SHIBUSDT", "DOTUSDT", "BTTUSDT", "LINKUSDT", "ALGOUSDT", "AVAXUSDT",
           "XLMUSDT", "NEARUSDT", "LTCUSDT", "CHZUSDT", "POLUSDT", "GRTUSDT",
           "LRCUSDT", "ARBUSDT", "UNIUSDT", "GALAUSDT", "INJUSDT", "TRXUSDT", "CRVUSDT",
           "ANKRUSDT", "NMRUSDT", "WOOUSDT", "MANAUSDT", "AAVEUSDT", "QNTUSDT", "BCHUSDT",
           "SUSHIUSDT", "APEUSDT", "ZRXUSDT", "ETCUSDT", "KSMUSDT", "SANDUSDT", "IMXUSDT", 
           "1INCHUSDT", "OPUSDT", "ATOMUSDT", "POWRUSDT", "AXSUSDT", "YFIUSDT", 
           "SNXUSDT", "MKRUSDT", "STORJUSDT", "GNOUSDT", "BATUSDT", "REQUSDT", "COMPUSDT", 
           "XTZUSDT", "BNTUSDT", "ENJUSDT", "EOSUSDT"]

# Date range
start_str = "2020-01-01"
end_str = "2025-01-01"

# Helper to get historical data from Binance
def get_binance_ohlcv(symbol, interval, start_str, end_str):
    df = pd.DataFrame(client.get_historical_klines(symbol, interval, start_str, end_str))
    if df.empty:
        return None
    df = df.iloc[:, 0:6]
    df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df.astype(float)
    return df

# Create a dictionary of DataFrames for each ticker
data = {}
for ticker in tickers:
    try:
        print(f"Downloading data for {ticker}...")
        df = get_binance_ohlcv(ticker, Client.KLINE_INTERVAL_1DAY, start_str, end_str)
        if df is not None:
            data[ticker] = df
        time.sleep(0.3)  # To avoid hitting Binance rate limits
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")

In [ ]:
import yfinance as yf
btc_df = data['BTCUSDT'][['close']].rename(columns={'close': 'BTC'})
btc_df.index = pd.to_datetime(btc_df.index)
spy = yf.download("SPY", start="2020-01-01", end="2025-01-01")
spy = spy[['Close']].rename(columns={'Close': 'SPY'})
spy.index = pd.to_datetime(spy.index)
spy = spy["SPY"]

In [ ]:
# Date range
start_str = "2022-01-01"
end_str = "2025-01-01"

# Helper to get historical data from Binance
def get_binance_ohlcv(symbol, interval, start_str, end_str):
    df = pd.DataFrame(client.get_historical_klines(symbol, interval, start_str, end_str))
    if df.empty:
        return None
    df = df.iloc[:, 0:6]
    df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df.astype(float)
    return df

# Create a dictionary of DataFrames for each ticker
data_2022 = {}
for ticker in tickers:
    try:
        print(f"Downloading data for {ticker}...")
        df = get_binance_ohlcv(ticker, Client.KLINE_INTERVAL_1DAY, start_str, end_str)
        if df is not None:
            data_2022[ticker] = df
        time.sleep(0.3)  # To avoid hitting Binance rate limits
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")

In [ ]:
import yfinance as yf
btc_df = data_2022['BTCUSDT'][['close']].rename(columns={'close': 'BTC'})
btc_df.index = pd.to_datetime(btc_df.index)
spy_2022 = yf.download("SPY", start="2022-01-01", end="2025-01-01")
spy_2022 = spy_2022[['Close']].rename(columns={'Close': 'SPY'})
spy_2022.index = pd.to_datetime(spy_2022.index)
spy_2022 = spy_2022["SPY"]

In [ ]:
import numpy as np
import pandas as pd

def strategy1_momentum_trend_confirmation(df):
    df = df.copy()

    # --- RSI ---
    delta = df['close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # --- 50-day SMA ---
    df['SMA50'] = df['close'].rolling(50).mean()

    # --- Momentum (20-day return) ---
    df['momentum'] = df['close'] / df['close'].shift(20) - 1

    # --- Volatility filter ---
    df['volatility'] = df['close'].rolling(20).std()
    df['vol_filter'] = df['volatility'] > df['volatility'].rolling(100).mean()

    # --- Combined signal rules ---
    df['signal'] = 0
    df.loc[
        (df['momentum'] > 0.10) & 
        (df['close'] > df['SMA50']) & 
        (df['RSI'] < 80) & 
        (df['vol_filter']), 'signal'
    ] = 1

    df.loc[
        (df['RSI'] < 40) | (df['close'] < df['SMA50']),
        'signal'
    ] = -1

    # --- Position tracking ---
    position = 0
    positions = []
    for sig in df['signal']:
        if sig == 1 and position == 0:
            position = 1
        elif sig == -1 and position == 1:
            position = 0
        positions.append(position)
    df['positions'] = positions

    # --- Strength column (used in capital-constrained backtests) ---
    df['strength'] = 0.0
    df.loc[df['signal'] == 1, 'strength'] = df.loc[df['signal'] == 1, 'momentum']

    return df


def strategy2_breakout_volume(df):
    df = df.copy()
    # 20-day high
    df['roll20_high'] = df['high'].rolling(20).max()
    # 20-day average volume
    df['roll20_vol_avg'] = df['volume'].rolling(20).mean()

    # Calculate ATR(14)
    high_low = df['high'] - df['low']
    high_close_prev = (df['high'] - df['close'].shift()).abs()
    low_close_prev = (df['low'] - df['close'].shift()).abs()
    tr = pd.concat([high_low, high_close_prev, low_close_prev], axis=1).max(axis=1)
    df['ATR'] = tr.rolling(window=14).mean()
    
    df['signal'] = 0
    
    # Entry condition: price breaks above 20-day high AND volume > 1.5 * avg volume
    df.loc[(df['close'] > df['roll20_high'].shift(1)) & (df['volume'] > 1.5 * df['roll20_vol_avg'].shift(1)), 'signal'] = 1
    
    # Implement trailing stop based on ATR:
    df['peak'] = df['close'].cummax()
    df['trailing_stop'] = df['peak'] - 3 * df['ATR']
    
    # Exit condition: close < trailing stop
    df.loc[df['close'] < df['trailing_stop'], 'signal'] = -1

    # Positions
    position = 0
    positions = []
    for sig in df['signal']:
        if sig == 1 and position == 0:
            position = 1
        elif sig == -1 and position == 1:
            position = 0
        positions.append(position)
    df['positions'] = positions

    # Strength: volume / rolling avg volume on buy signals; else 0
    df['strength'] = 0.0
    df.loc[df['signal'] == 1, 'strength'] = df.loc[df['signal'] == 1, 'volume'] / df.loc[df['signal'] == 1, 'roll20_vol_avg']

    return df



def strategy3_volatility_breakout(df):
    df = df.copy()
    # Bollinger Bands (20-day, 2 std)
    df['roll20_mean'] = df['close'].rolling(20).mean()
    df['roll20_std'] = df['close'].rolling(20).std()
    df['BB_upper'] = df['roll20_mean'] + 2 * df['roll20_std']
    df['BB_lower'] = df['roll20_mean'] - 2 * df['roll20_std']

    # Signal: buy when close crosses above BB_upper, sell when close crosses below BB_lower
    df['signal'] = 0
    prev_close = df['close'].shift(1)
    df.loc[(df['close'] > df['BB_upper']) & (prev_close <= df['BB_upper']), 'signal'] = 1
    df.loc[(df['close'] < df['BB_lower']) & (prev_close >= df['BB_lower']), 'signal'] = -1

    # Positions
    position = 0
    positions = []
    for sig in df['signal']:
        if sig == 1 and position == 0:
            position = 1
        elif sig == -1 and position == 1:
            position = 0
        positions.append(position)
    df['positions'] = positions

    # Strength: distance above BB_upper on buy signals; else 0
    df['strength'] = 0.0
    df.loc[df['signal'] == 1, 'strength'] = df.loc[df['signal'] == 1, 'close'] - df.loc[df['signal'] == 1, 'BB_upper']

    return df


In [ ]:
def backtest_signals(df):
    """
    df: DataFrame with 'close', 'high', 'low', and 'positions' columns.
    'positions' = 1 means holding, 0 means no position.
    Returns a summary dict with metrics and list of trades.
    """
    df = df.copy()
    df['position_shift'] = df['positions'].shift(1).fillna(0)

    trades = []
    entry_idx = None

    for i in range(len(df) - 1):  # avoid index out of bounds for i+1
        # Entry signal detected
        if df['positions'].iloc[i] == 1 and df['position_shift'].iloc[i] == 0:
            entry_idx = i + 1  # trade happens next day

        # Exit signal detected
        elif df['positions'].iloc[i] == 0 and df['position_shift'].iloc[i] == 1:
            exit_idx = i + 1  # trade happens next day
            if entry_idx is not None and exit_idx < len(df):
                entry_price = df['high'].iloc[entry_idx]
                exit_price = df['low'].iloc[exit_idx]
                entry_date = df.index[entry_idx]
                exit_date = df.index[exit_idx]
                ret = (exit_price - entry_price) / entry_price
                hold_time = (exit_date - entry_date).days
                trades.append({
                    'entry_date': entry_date,
                    'exit_date': exit_date,
                    'return': ret,
                    'hold_time_days': hold_time
                })
                entry_idx = None

    # If currently in a position at the end, close it on the next available day
    if entry_idx is not None and entry_idx < len(df):
        entry_price = df['high'].iloc[entry_idx]
        exit_price = df['low'].iloc[-1]
        entry_date = df.index[entry_idx]
        exit_date = df.index[-1]
        ret = (exit_price - entry_price) / entry_price
        hold_time = (exit_date - entry_date).days
        trades.append({
            'entry_date': entry_date,
            'exit_date': exit_date,
            'return': ret,
            'hold_time_days': hold_time
        })

    # Build daily equity curve
    equity = [1.0]
    for i in range(1, len(df)):
        if df['positions'].iloc[i - 1] == 1:
            daily_ret = (df['close'].iloc[i] - df['close'].iloc[i - 1]) / df['close'].iloc[i - 1]
            equity.append(equity[-1] * (1 + daily_ret))
        else:
            equity.append(equity[-1])
    equity = pd.Series(equity, index=df.index)

    # Max drawdown
    rolling_max = equity.cummax()
    drawdown = (equity - rolling_max) / rolling_max
    max_drawdown = drawdown.min()

    total_trades = len(trades)
    avg_return = np.mean([t['return'] for t in trades]) if trades else 0
    avg_hold = np.mean([t['hold_time_days'] for t in trades]) if trades else 0

    years = (df.index[-1] - df.index[0]).days / 365.25
    trades_per_year = total_trades / years if years > 0 else 0

    return {
        'total_trades': total_trades,
        'average_return': avg_return,
        'average_hold_days': avg_hold,
        'max_drawdown': max_drawdown,
        'trades_per_year': trades_per_year,
        'equity_curve': equity,
        'trades': trades
    }

In [ ]:
def aggregate_multi_crypto_stats(data_dict, strategy_func):
    """
    data_dict: dict of ticker -> DataFrame
    strategy_func: one of strategy1, strategy2, strategy3 functions that add 'positions' column
    
    Returns aggregated stats across all cryptos.
    """
    from collections import defaultdict
    
    trade_counts_per_day = defaultdict(int)
    all_trades = []
    
    total_equity = None
    
    for ticker, df in data_dict.items():
        strat_df = strategy_func(df)
        stats = backtest_signals(strat_df)
        all_trades.extend(stats['trades'])
        
        # Count active trades per day
        for i, pos in enumerate(strat_df['positions']):
            if pos == 1:
                trade_counts_per_day[strat_df.index[i]] += 1
        
        # Sum equity curves by aligning on dates
        if total_equity is None:
            total_equity = stats['equity_curve']
        else:
            total_equity = total_equity.add(stats['equity_curve'], fill_value=0)
    
    # Average number of active trades per day
    daily_active_counts = pd.Series(trade_counts_per_day)
    avg_active = daily_active_counts.mean()
    max_active = daily_active_counts.max()
    
    total_trades = len(all_trades)
    avg_return = np.mean([t['return'] for t in all_trades]) if all_trades else 0
    avg_hold = np.mean([t['hold_time_days'] for t in all_trades]) if all_trades else 0
    
    # Calculate max drawdown on combined equity (average of cryptos)
    if total_equity is not None:
        total_equity /= len(data_dict)  # average equity across cryptos
        rolling_max = total_equity.cummax()
        drawdown = (total_equity - rolling_max) / rolling_max
        max_drawdown = drawdown.min()
    else:
        max_drawdown = 0
    
    # Years based on combined date range
    all_dates = pd.concat([df.index.to_series() for df in data_dict.values()])
    total_days = (all_dates.max() - all_dates.min()).days
    years = total_days / 365.25 if total_days > 0 else 1
    trades_per_year = total_trades / years if years > 0 else 0
    
    return {
        'total_trades': total_trades,
        'average_return': avg_return,
        'average_hold_days': avg_hold,
        'max_drawdown': max_drawdown,
        'trades_per_year': trades_per_year,
        'average_active_trades_per_day': avg_active,
        'max_active_trades': max_active
    }


In [ ]:
agg_stats = aggregate_multi_crypto_stats(data, strategy1_momentum_trend_confirmation)
print(agg_stats)
agg_stats = aggregate_multi_crypto_stats(data, strategy2_breakout_volume)
print(agg_stats)
agg_stats = aggregate_multi_crypto_stats(data, strategy3_volatility_breakout)
print(agg_stats)

In [ ]:
agg_stats = aggregate_multi_crypto_stats(data_2022, strategy1_momentum_trend_confirmation)
print(agg_stats)
agg_stats = aggregate_multi_crypto_stats(data_2022, strategy2_breakout_volume)
print(agg_stats)
agg_stats = aggregate_multi_crypto_stats(data_2022, strategy3_volatility_breakout)
print(agg_stats)

In [ ]:
def backtest_with_max_active(data_dict, strategy_func, max_active=5):
    """
    Executes trades 1 day after signals:
    - Buys at next day's high
    - Sells at next day's low
    """
    all_trades = []

    # First, apply strategy and calculate strength
    processed = {}
    for ticker, df in data_dict.items():
        strat_df = strategy_func(df).copy()
        if 'RSI' in strat_df.columns:
            strat_df['strength'] = strat_df['RSI']
        elif 'volume' in strat_df.columns and 'roll20_vol_avg' in strat_df.columns:
            strat_df['strength'] = strat_df['volume'] / strat_df['roll20_vol_avg']
        elif 'BB_upper' in strat_df.columns:
            strat_df['strength'] = strat_df['close'] - strat_df['BB_upper']
        else:
            strat_df['strength'] = 0
        processed[ticker] = strat_df

    # Collect all signals
    signals = []
    for ticker, df in processed.items():
        for date, row in df.iterrows():
            signals.append({
                'date': date,
                'ticker': ticker,
                'signal': row['signal'],
                'strength': row['strength'],
                'close': row['close'],
                'high': row.get('high', row['close']),
                'low': row.get('low', row['close']),
            })
    signals_df = pd.DataFrame(signals).sort_values('date')
    dates = sorted(set(signals_df['date']))

    active_trades = {}
    equity = []
    capital = 1.0
    capital_per_trade = capital / max_active

    for i in range(len(dates) - 1):  # avoid out-of-bounds for i+1
        current_date = dates[i]
        next_date = dates[i + 1]
        today_signals = signals_df[signals_df['date'] == current_date]

        # Process exits
        exit_tickers = today_signals[
            (today_signals['signal'] == -1) & 
            (today_signals['ticker'].isin(active_trades.keys()))
        ]['ticker'].tolist()

        for t in exit_tickers:
            df = processed[t]
            if next_date in df.index:
                exit_price = df.loc[next_date, 'low']
                entry_price = active_trades[t]['entry_price']
                ret = (exit_price - entry_price) / entry_price
                capital += capital_per_trade * ret
                all_trades.append({
                    'ticker': t,
                    'entry_date': active_trades[t]['entry_date'],
                    'exit_date': next_date,
                    'entry_price': entry_price,
                    'exit_price': exit_price,
                    'return': ret
                })
                del active_trades[t]

        # Process entries
        free_slots = max_active - len(active_trades)
        buy_signals = today_signals[
            (today_signals['signal'] == 1) & 
            (~today_signals['ticker'].isin(active_trades.keys()))
        ]

        if free_slots > 0 and len(buy_signals) > 0:
            buy_signals = buy_signals.sort_values('strength', ascending=False).head(free_slots)
            for _, row in buy_signals.iterrows():
                t = row['ticker']
                df = processed[t]
                if next_date in df.index:
                    entry_price = df.loc[next_date, 'high']
                    active_trades[t] = {
                        'entry_date': next_date,
                        'entry_price': entry_price
                    }

        # Compute daily return from open trades (approx, based on close-close)
        daily_return = 0
        for t, trade in active_trades.items():
            df = processed[t]
            if current_date in df.index and dates[i - 1] in df.index:
                today_close = df.loc[current_date, 'close']
                prev_close = df.loc[dates[i - 1], 'close']
                ret = (today_close - prev_close) / prev_close
                daily_return += capital_per_trade * ret

        capital += daily_return
        equity.append({'date': current_date, 'capital': capital, 'open_trades': len(active_trades)})

    equity_df = pd.DataFrame(equity).set_index('date')
    cum_return = capital - 1
    equity_series = equity_df['capital']
    rolling_max = equity_series.cummax()
    drawdown = (equity_series - rolling_max) / rolling_max
    max_dd = drawdown.min()

    return {
        'equity_curve': equity_df,
        'cumulative_return': cum_return,
        'max_drawdown': max_dd,
        'max_active_trades': max(equity_df['open_trades']),
        'trades': all_trades
    }


In [ ]:
import matplotlib.pyplot as plt
# --- Run backtest ---
results = backtest_with_max_active(data, strategy1_momentum_trend_confirmation, max_active=5)
strategy_curve = results['equity_curve'][['capital']].rename(columns={'capital': 'Strategy 1'})

# --- Merge all series ---
merged = pd.concat([strategy_curve, btc_df, spy], axis=1, join='inner').dropna()

normalized = merged / merged.iloc[0]

# --- Plot ---
plt.figure(figsize=(14, 7))
plt.plot(normalized.index, normalized['Strategy 1'], label='Strategy 1', linewidth=2)
plt.plot(normalized.index, normalized['BTC'], label='Bitcoin (BTC)', linewidth=2, linestyle='--')
plt.plot(normalized.index, normalized['SPY'], label='SPY (S&P 500)', linewidth=2, linestyle=':')
plt.title("Relative Performance: Strategy vs Bitcoin vs SPY")
plt.ylabel("Normalized Return (start = 1.0)")
plt.xlabel("Date")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Print summary ---
print(f"Cumulative Return: {results['cumulative_return']*100:.2f}%")
print(f"Max Drawdown: {results['max_drawdown']*100:.2f}%")
print(f"Max Active Trades: {results['max_active_trades']}")

In [ ]:
pd.DataFrame(results["trades"])

In [ ]:
results = backtest_with_max_active(data, strategy2_breakout_volume, max_active=5)
strategy_curve = results['equity_curve'][['capital']].rename(columns={'capital': 'Strategy 1'})

# --- Merge all series ---
merged = pd.concat([strategy_curve, btc_df, spy], axis=1, join='inner').dropna()

normalized = merged / merged.iloc[0]

# --- Plot ---
plt.figure(figsize=(14, 7))
plt.plot(normalized.index, normalized['Strategy 1'], label='Strategy 1', linewidth=2)
plt.plot(normalized.index, normalized['BTC'], label='Bitcoin (BTC)', linewidth=2, linestyle='--')
plt.plot(normalized.index, normalized['SPY'], label='SPY (S&P 500)', linewidth=2, linestyle=':')

plt.title('Backtest Equity Curve (strategy2_breakout_volume)')
plt.xlabel('Date')
plt.ylabel('Capital')
plt.legend()
plt.grid(True)
plt.show()

# --- Print summary ---
print(f"Cumulative Return: {results['cumulative_return']*100:.2f}%")
print(f"Max Drawdown: {results['max_drawdown']*100:.2f}%")
print(f"Max Active Trades: {results['max_active_trades']}")

In [ ]:
pd.DataFrame(results["trades"])

In [ ]:
results = backtest_with_max_active(data, strategy3_volatility_breakout, max_active=5)
strategy_curve = results['equity_curve'][['capital']].rename(columns={'capital': 'Strategy 1'})

# --- Merge all series ---
merged = pd.concat([strategy_curve, btc_df, spy], axis=1, join='inner').dropna()

normalized = merged / merged.iloc[0]

# --- Plot ---
plt.figure(figsize=(14, 7))
plt.plot(normalized.index, normalized['Strategy 1'], label='Strategy 1', linewidth=2)
plt.plot(normalized.index, normalized['BTC'], label='Bitcoin (BTC)', linewidth=2, linestyle='--')
plt.plot(normalized.index, normalized['SPY'], label='SPY (S&P 500)', linewidth=2, linestyle=':')

plt.title('Backtest Equity Curve (strategy3_volatility_breakout)')
plt.xlabel('Date')
plt.ylabel('Capital')
plt.legend()
plt.grid(True)
plt.show()

# --- Print summary ---
print(f"Cumulative Return: {results['cumulative_return']*100:.2f}%")
print(f"Max Drawdown: {results['max_drawdown']*100:.2f}%")
print(f"Max Active Trades: {results['max_active_trades']}")

In [ ]:
pd.DataFrame(results["trades"])

In [ ]:
import matplotlib.pyplot as plt
# --- Run backtest ---
results = backtest_with_max_active(data_2022, strategy1_momentum_trend_confirmation, max_active=5)
strategy_curve = results['equity_curve'][['capital']].rename(columns={'capital': 'Strategy 1'})

# --- Merge all series ---
merged = pd.concat([strategy_curve, btc_df, spy_2022], axis=1, join='inner').dropna()

normalized = merged / merged.iloc[0]

# --- Plot ---
plt.figure(figsize=(14, 7))
plt.plot(normalized.index, normalized['Strategy 1'], label='Strategy 1', linewidth=2)
plt.plot(normalized.index, normalized['BTC'], label='Bitcoin (BTC)', linewidth=2, linestyle='--')
plt.plot(normalized.index, normalized['SPY'], label='SPY (S&P 500)', linewidth=2, linestyle=':')
plt.title("Relative Performance: Strategy vs Bitcoin vs SPY")
plt.ylabel("Normalized Return (start = 1.0)")
plt.xlabel("Date")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Print summary ---
print(f"Cumulative Return: {results['cumulative_return']*100:.2f}%")
print(f"Max Drawdown: {results['max_drawdown']*100:.2f}%")
print(f"Max Active Trades: {results['max_active_trades']}")

In [ ]:
import matplotlib.pyplot as plt
# --- Run backtest ---
results = backtest_with_max_active(data_2022, strategy2_breakout_volume, max_active=5)
strategy_curve = results['equity_curve'][['capital']].rename(columns={'capital': 'Strategy 1'})

# --- Merge all series ---
merged = pd.concat([strategy_curve, btc_df, spy_2022], axis=1, join='inner').dropna()

normalized = merged / merged.iloc[0]

# --- Plot ---
plt.figure(figsize=(14, 7))
plt.plot(normalized.index, normalized['Strategy 1'], label='Strategy 1', linewidth=2)
plt.plot(normalized.index, normalized['BTC'], label='Bitcoin (BTC)', linewidth=2, linestyle='--')
plt.plot(normalized.index, normalized['SPY'], label='SPY (S&P 500)', linewidth=2, linestyle=':')
plt.title("Relative Performance: Strategy vs Bitcoin vs SPY")
plt.ylabel("Normalized Return (start = 1.0)")
plt.xlabel("Date")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Print summary ---
print(f"Cumulative Return: {results['cumulative_return']*100:.2f}%")
print(f"Max Drawdown: {results['max_drawdown']*100:.2f}%")
print(f"Max Active Trades: {results['max_active_trades']}")

In [ ]:
pd.DataFrame(results["trades"])

In [ ]:
import matplotlib.pyplot as plt
# --- Run backtest ---
results = backtest_with_max_active(data_2022, strategy3_volatility_breakout, max_active=5)
strategy_curve = results['equity_curve'][['capital']].rename(columns={'capital': 'Strategy 1'})

# --- Merge all series ---
merged = pd.concat([strategy_curve, btc_df, spy_2022], axis=1, join='inner').dropna()

normalized = merged / merged.iloc[0]

# --- Plot ---
plt.figure(figsize=(14, 7))
plt.plot(normalized.index, normalized['Strategy 1'], label='Strategy 1', linewidth=2)
plt.plot(normalized.index, normalized['BTC'], label='Bitcoin (BTC)', linewidth=2, linestyle='--')
plt.plot(normalized.index, normalized['SPY'], label='SPY (S&P 500)', linewidth=2, linestyle=':')
plt.title("Relative Performance: Strategy vs Bitcoin vs SPY")
plt.ylabel("Normalized Return (start = 1.0)")
plt.xlabel("Date")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Print summary ---
print(f"Cumulative Return: {results['cumulative_return']*100:.2f}%")
print(f"Max Drawdown: {results['max_drawdown']*100:.2f}%")
print(f"Max Active Trades: {results['max_active_trades']}")